# Testing methods of Mishra et al. 2022

[link to paper](http://journals.sagepub.com/doi/10.1177/0272989X211044697)

**Note about dataset**

Measure of clinical utility is the standardized net benefit.

* $ R \equiv $ the risk threshold for recommending an intervention.
* $ \pi $ is the prevalence of the outcome
* $\text{TPR}_R \equiv$ the true positive rate when the model risk threshold is $R$. AKA the sensitivity or the hit rate.
* $\text{FPR}_R \equiv$ the same for the false positive rate. AKA 1 - specificity or the probability of a false alarm.
* The ratio $R/(1-R)$ represents the ratio of benefits and harms of recommending an intervention. In this case I would assume this means the benefits vs. harms of screening a patient.

$$ \text{sNB}_R = \text{TPR}_R - \frac{R}{1-R}\frac{1-\pi}{\pi} \text{FPR}_R$$

The advantage of this measure of clinical utility is that its maximum is 1 when the TPR=1 and FPR=0. It is an "opt-in" formulation that assumes the default should be not treating or screening the patient. Other measures of clinical utility include the net benefit and relative utility.

The behavior of the sNB is that as the risk becomes large, it penalizes a higher false positive rate. As the prevalence becomes higher, it penalizes false positives less. For the rare-disease case, we might expect that the risk is high of non-detection, but that the prevalence is somewhat low. So perhaps we want the threshold $R$ closer to 0 to indicate that the our threshold for screening is low.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [12]:
from sklearn.datasets import load_breast_cancer

## Read and process the data set

To avoid duplicating the available data, you must download the data yourself and place it in the correct directory. Find the data at [this link](https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008).

In [17]:
bc_data = load_breast_cancer()

In [23]:
print(bc_data['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [18]:
df = pd.DataFrame(data=bc_data.data, 
                  columns=bc_data.feature_names)

In [26]:
labels = pd.DataFrame(bc_data['target'])

The labels indicate whether the case is malignant (0) or benign (1). 

In [27]:
labels.value_counts()

1    357
0    212
dtype: int64

## Artificially split the data set

Rather than randomly splitting the data set, I will split using an artificial criterion to create some difference between the training set patient population and the target set, because this toy dataset is a relatively easy classification problem. I intentionally want a performance drop (still need to define that quantitatively) in the held-out data here, so that I can "fix" it by recalibration.